In [12]:
import os
import cv2
import numpy as  np
import matplotlib.pyplot as plt
from pathlib import Path
import kikuchipy as kp
from orix.quaternion import Rotation
from openECCI import io, rkp
from openECCI.ctf import file_reader as ctf_reader

plt.rcParams.update({
    "figure.figsize": (7, 7),
    "font.size": 9,
})

In [13]:
# Read IPF map Image to be aligned
# Path to fcc_Fe .ctf file containing the EBSD euler angles
fe_xmap_path = r"D:\OneDrive - Monash University\2018-2019 MCEM\02 Projects\13 EBSD n ECCI\20230213 tutorial manuscript\supplementary"
fe_xmap_file = r"20kv_26nA_15mm WD_4x4 bin_fcc_Fe Map.ctf"
fe_xmap_fname = os.path.join(fe_xmap_path, fe_xmap_file)
fe_xmap = ctf_reader(fe_xmap_fname)
ipf_map = rkp.get_xmap_image(fe_xmap, phase_name="Iron fcc", overlay="BC").astype('float32')

# Reference Reference image or Ideal image
sem_path = r"D:\OneDrive - Monash University\2018-2019 MCEM\02 Projects\13 EBSD n ECCI\20231024 ECCI"
sem_file = r"01_steel overview.tif"
sem_fname = os.path.join(sem_path, sem_file)
sem = cv2.imread(sem_fname).astype('float32')

sem_norm = cv2.cvtColor(sem, cv2.COLOR_BGR2RGB)
sem_norm /= 255
ipf_image = cv2.cvtColor(ipf_map, cv2.COLOR_BGR2RGB)
height, width, _ = sem_norm.shape
print(height, width)

2188 3072


In [16]:
%matplotlib qt
# display the interactive EBSD IPF map to select correlation points
# Create a function to handle mouse click events
def onclick(event):
    if event.dblclick:
        x, y = int(event.xdata), int(event.ydata)  # Get the pixel coordinates from the click event
        points.append((x, y))  # Store the selected point
        labels.append(len(points))  # Prompt for a label

        # Update the plot to show the selected point and label
        ax.plot(x, y, 'ro', markersize=5)
        ax.text(x, y, labels[-1], fontsize=12, color='red')
        plt.draw()

# Initialize lists to store selected points and their labels
points = []
labels = []

# Create a figure and display the image
fig, ax = plt.subplots()
ax.imshow(ipf_image)
ax.set_title("Click on the image to select points.")

# Connect the mouse click event to the function
cid = fig.canvas.mpl_connect('button_press_event', onclick)

# Display the image and allow interactive selection
plt.show()



In [17]:
# display the interactive SEM image to select correlation points
# Create a function to handle mouse click events
def onclick2(event):
    if event.dblclick:
        x2, y2 = int(event.xdata), int(event.ydata)  # Get the pixel coordinates from the click event
        points2.append((x2, y2))  # Store the selected point
        labels2.append(len(points2))  # Prompt for a label
        

        # Update the plot to show the selected point and label
        ax2.plot(x2, y2, 'ro', markersize=5)
        ax2.text(x2, y2, labels2[-1], fontsize=12, color='red')
        plt.draw()
        plt.pause(0.0001)

# Initialize lists to store selected points and their labels
points2 = []
labels2 = []

# Create a figure and display the image
fig2, ax2 = plt.subplots()
ax2.imshow(sem_norm, cmap="gray")
ax2.set_title("Click on the image to select points.")

# Connect the mouse click event to the function
cid = fig2.canvas.mpl_connect('button_press_event', onclick2)

# Display the image and allow interactive selection
plt.show()



In [ ]:
# When need to save the alignment points for future use, run this cell
points2 = np.array(points2, dtype=np.float32)
points1 = np.array(points, dtype=np.float32)
io.save_align_points(points1, points2)

In [8]:
# When need to load the saved alignment points for future use, run this cell
[points1, points2] = io.load_prev_align_points()
len(points1)

24

In [9]:
pts_sem = np.array(points2, dtype=np.float32)
pts_ipf = np.array(points1, dtype=np.float32)

homography, mask = cv2.findHomography(pts_ipf, pts_sem, 0)
warp_src = cv2.warpPerspective(ipf_map, homography, (width, height))
inverse_transformation_matrix = np.linalg.inv(homography)

In [18]:
%matplotlib qt
alpha = 0.6
beta = (1.0 - alpha)
dst_warp_blended = cv2.addWeighted(sem_norm, alpha, warp_src, beta, 0.0)
# cv2.namedWindow('result',cv2.WINDOW_NORMAL)
# cv2.imshow('result', dst_warp_blended)
fig = plt.subplots(figsize=(10, 8))
plt.imshow(dst_warp_blended)
plt.axis("off")
plt.tight_layout()
# plt.savefig("overlay_ipf.png", dpi=300)
# plt.show()